In [169]:
from IPython.display import display
from ipywidgets import widgets
from os.path import join
import guild.ipy as guild

def _run_time_seconds(attrs):
    started = attrs["started"]
    stopped = attrs["stopped"]
    return (stopped-started)//1e6

def _runs_data(last_n_runs):
    import collections
    data = collections.defaultdict(dict)
    scalars = runs.scalars()[:last_n_runs].filter(items=["run","last_val",])
    for run, loss in zip(scalars["run"], scalars["last_val"]):
        run_data = data[run]
        run_data["loss"] = loss
    
    for run in guild.runs()[:last_n_runs]["run"]:
        run = run.run
        attrs = dict(run.iter_attrs())
        run_data = data[run.id]
        run_data["flags"] = attrs["flags"]
        run_data["time"] = _run_time_seconds(attrs)

    return data

def _image_widget(img_path, scale=1):
    return widgets.Image(
        value=open(img_path, 'rb').read(),
        width=int(660*scale),
        height=int(400*scale),
    )

def _text_widget(key, value):
    return widgets.HTML(f'<span style="white-space: nowrap">{key}: {value}</span>')

def _render_view(run, runs_data):
    run_data = runs_data[run.id]

    stats = [
        ("run", run.id),
        ("loss", run_data['loss']),
        ("time", run_data['time']),
    ] +  list(run_data['flags'].items())

    stats_box = widgets.HBox([
        widgets.VBox([_text_widget(key, value) for key, value in stats]),
    ])

    run_dir = run.dir
    image_box = widgets.HBox([
        widgets.VBox([_image_widget(img_path, scale) for img_path, scale in [
            (join(run_dir, "content_img.png"), 0.6),
            (join(run_dir, "style_img.png"), 0.6),
        ]]),
        _image_widget(join(run_dir, "generated_img.png"), 1.2),
    ])

    row = widgets.HBox([stats_box, image_box])
    display(row)

runs = guild.runs()
run_series = runs["run"]
last_n_runs = 10 # choose how many runs to display data for
runs_data = _runs_data(last_n_runs)
for run in run_series[:last_n_runs]:
    _render_view(run.run, runs_data)